# Creating an index fund

The goal of this project is the definition of an index fund, following the Dow Jones. That is, we want to select few stocks of the index, together with weights, that show a similar behavior to the overall index.

We start with price data of all the Dow Jones stocks from 2016. From the averages prices, we define weights of the stocks to be used

## Loading the price data

The data is provided in a file using comma-separated values and three columns:

In [ ]:
;head dowjones2016.csv

Julia provides a function to read csv files into arrays:

But we will use the DataFrames package for easier processing.

In [ ]:
using DataFrames

In [ ]:
df = readtable("dowjones2016.csv")
df[1:4, :]

We can now access the columns by name:

In [ ]:
df[:price]

Let's compute mean prices for the stocks, using a groupby-and-aggregate approach.

In [ ]:
using Statistics

In [ ]:
avg = by(df, :symbol, d -> DataFrame(avgprice = mean(d[:price])))
avg[1:4, :]

We can now use these averages to compute weights.

In [ ]:
weights = DataFrame(symbol = avg[:symbol],
                    weight = avg[:avgprice] / sum(avg[:avgprice]))
weights[1:4, :]

In [ ]:
joined = join(df, weights, on=:symbol)
joined[1:4, :]

In [ ]:
joined[:contribution] = joined[:weight] .* joined[:price]
joined[1:4, :]

In [ ]:
index = by(joined, :date, d -> DataFrame(value = sum(d[:contribution])))
index[1:4, :]

In [ ]:
# two-way table with symbols as columns
#                    rows   columns  data
prices = unstack(df, :date, :symbol, :price)
prices[1:4, 1:4]

## Visualization the time series

In [ ]:
using Plots      # general plotting
using StatPlots  # for DataFrames integration
gr()

In [ ]:
with(grid=false, legend=false, xticks=false, ylim=(0,300)) do
    plot(df, :date, :price, group=:symbol, color=:grey, alpha=0.4)
    plot!(index, :date, :value, linewidth=2)
end

## Picking stocks

We know come to the decision problem, where we want to pick a small subset of the stocks together with some weights, such that this portfolio has a similar behavior to our overall Dow Jones index.

The model is based on a linear regression over the time series, but we minimize the loss using the L1-norm (absolute value), and allow only a fixed number of weights to take nonzero variable.

A high-level mathematical model might look like this ($w$: weights, $P$: prices, $I$: value of index):

\begin{align*}
\text{minimize}   \quad & \lVert w^T P - I \rVert_1 \\
\text{subject to} \quad & \lVert w \rVert_0 \le K
\end{align*}

For the curious: this can be expressed as a [Mixed-Integer Linear Program](https://en.wikipedia.org/wiki/Integer_programming) in the following form:

\begin{align*}
\text{minimize}   \quad & \sum_d \Delta^+_d + \Delta^-_d & \\
\text{subject to} \quad & \sum_s P_{d,s} w_s = I_d + \Delta^+_d + \Delta^-_d & (\forall d) \\
                        & w_s \le p_s & (\forall s) \\
                        & \sum_s p_s \le K & \\
                        & w_s \ge 0, \quad p_s \in \{0,1\} & (\forall s) \\
                        & \Delta^+_d \ge 0, \quad \Delta^-_d \ge 0 &  (\forall d)
\end{align*}

Several Julia packages are devoted to this kind of optimization, such as [JuMP](https://github.com/JuliaOpt/JuMP.jl) and [Convex](https://github.com/JuliaOpt/Convex.jl) for modeling, solver backends like [Cbc](https://github.com/JuliaOpt/Cbc.jl) or [SCIP](https://github.com/SCIP-Interfaces/SCIP.jl) and [MathProgBase](https://github.com/JuliaOpt/MathProgBase.jl) as glue. See [JuliaOpt](http://www.juliaopt.org/) for an overview.

In [ ]:
using JuMP # modeling
using SCIP # solver backend

In [ ]:
# preparing data for indexing
syms = [Symbol(s) for s in weights[:symbol]]
days = 1:length(prices[:date])

@show size(syms) size(days);

We will formulate a model that should look quite close to the mathematical notation above.

Note the heavy use of Julia macros to define variables and constraints. The expressions are used as parsed by the Julia language and directly translated to the solver's internal form.

In [ ]:
function find_fund(maxstocks; timelimit=10.0, gaplimit=0.01, lastday=200)
    days = 1:lastday

    fund = Model(solver=SCIPSolver("limits/time", timelimit, "limits/gap", gaplimit))

    # decisions
    @variable(fund, pick[syms], Bin)    # is stock included?
    @variable(fund, weight[syms] ≥ 0)   # what part of the portfolio

    # auxiliary variables
    @variable(fund, Δ⁺[days] ≥ 0) # positive slack
    @variable(fund, Δ⁻[days] ≥ 0) # negative slack

    # fit to Dow Jones index
    for d in days
        @constraint(fund, sum(prices[d,s] * weight[s] for s in syms) == index[d, :value] + Δ⁺[d] - Δ⁻[d])
    end

    # can only use stock if picked
    for s in syms
        @constraint(fund, weight[s] ≤ pick[s])
    end
                
    # few stocks allowed
    @constraint(fund, sum(pick[s] for s in syms) ≤ maxstocks)
                          
    # minimize the absolute violation (L1 norm)
    @objective(fund, :Min, sum(Δ⁺[d] + Δ⁻[d] for d in days))
                            
                            
    status = solve(fund)
    @show status
    
    getvalue(weight)
end

In [ ]:
trainingdays = 100
sol = find_fund(3, timelimit=6, lastday=trainingdays)

In [ ]:
solfund = sum(sol[s] * prices[:, s] for s in syms);

In [ ]:
with(xticks=[0, trainingdays, length(days)], yticks=[]) do
    plot(index, :date, :value, label="Dow Jones")
    plot!(solfund, label="Index Fund")
end